<a href="https://colab.research.google.com/github/sag111/MedRecommendation/blob/main/extractive_approach/creating_tables/drugs_parsing_json_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Создание таблиц из json-файлов размеченных отзывов 



In [19]:
import json
import pandas as pd

## Импорт таблицы с упоминаниями лекарственных препаратов и соответствующих им стандартными названиями. Проверка корректности импорта

In [20]:
drugname_df = pd.read_excel('Drugname_2022.xlsx')
drugname_df

,mention,standart,ATC,description,comments,med_NNreview,med_1kADR
0,2 в 1 garnier чистая кожа актив,garnier 2 в 1 чистая кожа актив,NaN,NaN,Косметическое средство,NaN,563469
1,5 нок,5 нок,J01XX07,Nitroxoline,NaN,1383732,NaN
2,5- нок,5 нок,J01XX07,Nitroxoline,NaN,1383732,NaN
3,5нок,5 нок,J01XX07,Nitroxoline,NaN,3145688,NaN
4,"911 ""бодяга",911 бадяга,D11AX,Прочие препараты для лечения заболеваний кожи,NaN,NaN,2702981
...,...,...,...,...,...,...,...
2604,vicks active sinex,vicks active sinex,R01AA05,Oxymetazoline,NaN,NaN,2249637
2605,vitrum,vitrum,A11AA04,Поливитамины с микроэлементами,NaN,NaN,1373327/671606
2606,vitrum kids,vitrum,A11AA04,Поливитамины с микроэлементами,NaN,NaN,4283315
2607,zerosmoke,zerosmoke,N07BA03,Varenicline,NaN,NaN,1175393/1256045/1929179


## Определение вспомогательных функций 

In [21]:
def standartization(drug_list):    
    standart_drug_list = [0 for i in range(len(drug_list))]
    mention, standart = list(drugname_df.mention), list(drugname_df.standart)
    for i in range(len(drug_list)):
      if drug_list[i].lower() in mention:
        indices = [j for j in range(0, len(mention)) if mention[j]==drug_list[i].lower()]
        for ind in indices:
          standart_drug_list[i] = standart[ind]
    return standart_drug_list


def remove_duplicates(list_with_duplicates):
    list2 = []
    for item in list_with_duplicates:
      if item not in list2:
        list2.append(item)
    return list2


def top3_drugs(ordered_list_of_drugs):
    if len(ordered_list_of_drugs) > 3:
        return ordered_list_of_drugs[:3]
    else:
        return ordered_list_of_drugs


def count_frequency(list_for_count):
    dict_with_frequency = {}
    for elem in list_for_count:
        if elem not in dict_with_frequency:
            dict_with_frequency[elem] = 0    
        dict_with_frequency[elem] += 1
    return dict_with_frequency

## Парсинг json-файлов:
- cоздание для каждого упоминания лекарственного препарата количества BNE-Pos, ADE-Neg, Worse, NegatedADE, ADR и списка ADR
- создание для каждого симптома списка лекарственных препаратов 

In [22]:
with open("test.json", "r") as f:
    data = json.load(f)

names = []
bne_counts = []
ade_neg_counts = []
worse_counts = []
negated_ade_counts = []
adr_counts = []
adr_list = []
diseases = {}

for i in range(len(data)):
    text_id = data[i]['text_id']
    curr_name = []
    for entity in data[i]['entities'].values():
        bne_count = 0
        ade_neg_count = 0
        worse_count = 0
        negated_ade_count = 0
        adr_count = 0
        adr_text = 'Нет ADR'
        if "tag" in entity and "Context" in entity:          
            if 'Medication:MedTypeDrugname' in entity['tag'] and entity["Context"] == ["1"]:
                name = entity["text"]
            if entity["tag"] == ["Disease:DisTypeBNE-Pos"] and entity["Context"] == ["1"]:
                bne_count += 1
            if entity["tag"] == ['Disease:DisTypeADE-Neg'] and entity["Context"] == ["1"]:
                ade_neg_count += 1
            if entity["tag"] == ['Disease:DisTypeWorse'] and entity["Context"] == ["1"]:
                worse_count += 1
            if entity["tag"] == ['Disease:DisTypeNegatedADE'] and entity["Context"] == ["1"]:
                negated_ade_count += 1
            if entity["tag"] == ['ADR'] and entity["Context"] == ["1"]:
                adr_count += 1
            if entity["tag"] == ['ADR'] and 'MedDRA' in entity and entity["Context"] == ["1"]:
                adr_text = entity['MedDRA']
            if entity['tag'] == ['Disease:DisTypeIndication'] and 'MedDRA' in entity and entity['Context'] == ['1']:
                key = entity['MedDRA']
                if key not in diseases:
                    diseases[key] = []
                diseases[key].append(name)
        names.append(name)
        bne_counts.append(bne_count)
        ade_neg_counts.append(ade_neg_count)
        worse_counts.append(worse_count)
        negated_ade_counts.append(negated_ade_count)
        adr_counts.append(adr_count)
        adr_list.append(adr_text)

In [23]:
#print(sum(adr_counts))
#count = 0
#for elem in adr_list:
#    if elem != 'Нет ADR':
#        count += 1
#print(count)

## Создание таблицы препарат -- рейтинг -- список возможных ADR

### Создание таблицы из списков количества сущностей BNE-Pos, ADE-Neg, Worse, NegatedADE, ADR для каждого препарата

In [24]:
standart_names = standartization(names)

data = {
        'standart_names':standart_names, 
        'bne_counts':bne_counts, 
        'ade_neg_counts':ade_neg_counts,
        'worse_counts':worse_counts,
        'negated_ade_counts':negated_ade_counts,
        'adr_counts':adr_counts
        }

df_raw = pd.DataFrame(data)
df_drugs = df_raw.groupby('standart_names').sum()

### Добавление в таблицу количества упоминаний для каждого препарата 

In [25]:
df_for_mentions_count = df_raw.groupby('standart_names').count()
number = df_for_mentions_count['bne_counts']
df_drugs.insert(loc=len(df_drugs.columns), column='number_of_mentions', value=number) 

### Добавление списка возможных ADR каждого препарата. Список возможных ADR сделан в виде словаря, где для каждого варианта ADR (в том числе для их отсутствия) посчитано количество упоминаний возникновения такого ADR у препарата

In [26]:
data_adr = {
        'standart_names':standart_names, 
        'adr_list':adr_list
        }

df_adr = pd.DataFrame(data_adr)
df2_text = df_adr.groupby('standart_names').count() 
adr_list_df = list(df2_text['adr_list'])

In [27]:
pointer = 0
adr_list_text = [''] * len(df2_text)
for i in range(len(df2_text)):
    pointer += adr_list_df[i]
    adr_list_text[i] = count_frequency(adr_list[pointer-adr_list_df[i]+1:pointer+1])
df_drugs.insert(loc=len(df_drugs.columns), column='adr_list_text', value=adr_list_text)

### Вычисление рейтинга препаратов. Колонка neg_sum показывает сумму негативных упоминаний для препарата

In [28]:
df_drugs['neg_sum'] = df_drugs['ade_neg_counts'] + df_drugs['worse_counts'] + df_drugs['negated_ade_counts'] + df_drugs['adr_counts']
raiting = [''] * len(df_drugs)
for i in range(len(df_drugs)):
   if df_drugs.iloc[i]['bne_counts'] > df_drugs.iloc[i]['neg_sum']:
       if len(df_drugs.iloc[i]['adr_list_text']) > 1:
           raiting[i] = 'Хороший, но есть ADR'
       else:
           raiting[i] = 'Хороший'
   else:
       if df_drugs.iloc[i]['adr_counts'] > df_drugs.iloc[i]['negated_ade_counts'] and df_drugs.iloc[i]['adr_counts'] > df_drugs.iloc[i]['worse_counts'] and df_drugs.iloc[i]['adr_counts'] > df_drugs.iloc[i]['ade_neg_counts']:
           raiting[i] = 'Плохой: частные побочне реакции'
       elif df_drugs.iloc[i]['negated_ade_counts'] > df_drugs.iloc[i]['adr_counts'] and df_drugs.iloc[i]['negated_ade_counts'] > df_drugs.iloc[i]['worse_counts'] and df_drugs.iloc[i]['negated_ade_counts'] > df_drugs.iloc[i]['ade_neg_counts']:
           raiting[i] = 'Плохой: не помогает'
       else:
           raiting[i] = 'Плохой'
df_drugs.insert(loc=len(df_drugs.columns), column='raiting', value=raiting)

In [29]:
df_drugs_final = df_drugs.drop(['bne_counts', 'ade_neg_counts', 'worse_counts', 'negated_ade_counts', 'adr_counts', 'number_of_mentions', 'neg_sum'], axis=1)
df_drugs_final.columns = ['ADR', 'Raiting']
df_drugs_final

,ADR,Raiting
standart_names,,
cold balm,{'Нет ADR': 26},Хороший
evergetikon,{'Нет ADR': 8},Хороший
heplex breath,{'Нет ADR': 4},Плохой
агри,{'Нет ADR': 16},Плохой: не помогает
азалептин,"{'Нет ADR': 34, 'Гиперчувствительность': 3, 'С...",Плохой: частные побочне реакции
...,...,...
энцефабол,"{'Нет ADR': 118, 'Дисгевзия': 1, 'Сыпь, макуле...","Хороший, но есть ADR"
эразабан,{'Нет ADR': 40},Плохой: не помогает
эргоферон,"{'Нет ADR': 252, 'Сомнолентность|Брадифрения': 1}","Хороший, но есть ADR"


In [30]:
df_drugs_final.to_csv('drugs_table.csv')

In [31]:
df_drugs_final_aggregated = df_drugs_final.groupby('Raiting').count()
df_drugs_final_aggregated.columns = ['Count']
df_drugs_final_aggregated

,Count
Raiting,
Плохой,18
Плохой: не помогает,45
Плохой: частные побочне реакции,30
Хороший,53
"Хороший, но есть ADR",33


In [32]:
df_drugs_final_aggregated.to_csv('drugs_table_aggregated.csv')

## Создание датафрейма формата симптом -- список лучших препаратов

### Создание таблицы вида симптом - список всех нестандартизироавнных названий препаратов. 

In [33]:
data_diseases = {
    'disease':diseases.keys(),
    'drug': diseases.values()
}

df_diseases = pd.DataFrame(data_diseases)
df_diseases.set_index('disease', inplace=True)
df_diseases

,drug
disease,
Стресс,"[Тенотен, Валерианы экстракт, Валериана, Пусты..."
Низкое качество сна|Бессонница,"[Тенотен, Афобазол, Аквалор baby, Тенотен]"
Нервозность,"[Тенотен, Пустырник, Вечернее, интерферон, нас..."
Эритема глотки,"[Тонзилгон Н, Тонзилгон Н, Эргоферон]"
Боль в ротоглотке (орофарингеальная),"[Тонзилгон, Тонзилгон, Граммидин Нео с анестет..."
...,...
Гиперсекреторное слезоотделение|Ринорея,[ремантадин]
Расстройство поведения,"[Глицин, Глицин]"
Агрессия,[Глицисед]


### Далее преобразование данных - стандартизация названий препаратов, отбор препаратов только с хорошим рейтингом, сортировка по количеству упоминаний препарата, отбор максимум трех лучших. 

In [34]:
def only_good_raiting(list_of_drugs):
    new_list_of_drugs = []
    for drug in list_of_drugs:
        if df_drugs.loc[drug]['raiting'] in ['Хороший','Хороший, но есть ADR']:
            new_list_of_drugs.append(drug) 
    return new_list_of_drugs

def sort_drugs_on_review_number(list_of_drugs):
    dict_of_drugs = {}
    sorted_list = []
    for drug in list_of_drugs:
        dict_of_drugs[drug] = df_drugs.loc[drug]['number_of_mentions']
    sorted_tuples = sorted(dict_of_drugs.items(), key=lambda item: item[1])
    for i in range(len(sorted_tuples)):
        sorted_list.append(sorted_tuples[i][0])
    return sorted_list[::-1]


for key, value in diseases.items():
    diseases[key] = standartization(value)
    diseases[key] = remove_duplicates(diseases[key])
    diseases[key] = only_good_raiting(diseases[key])
    diseases[key] = sort_drugs_on_review_number(diseases[key])
    diseases[key] = top3_drugs(diseases[key])
    if diseases[key] == []:
        diseases[key] = 'Нет хороших лекарств'
    if diseases[key] != 'Нет хороших лекарств':
        diseases[key] = ', '.join(diseases[key])

In [35]:
data_diseases = {
    'disease':diseases.keys(),
    'drug': diseases.values()
}

df_diseases_final = pd.DataFrame(data_diseases)
df_diseases_final.head(50)

,disease,drug
0,Стресс,"ацикловир, глицин, валериана"
1,Низкое качество сна|Бессонница,аквалор беби
2,Нервозность,"глицин, валериана, интерферон"
3,Эритема глотки,"эргоферон, тонзилгон н"
4,Боль в ротоглотке (орофарингеальная),"кагоцел, ацикловир, эргоферон"
5,Боль в ротоглотке (орофарингеальная)|Орофаринг...,"афлубин, тонзилгон н"
6,Депрессия,Нет хороших лекарств
7,Нарушение засыпания,глицин
8,Ажитация,валериана
9,Повышенная температура тела,"афлубин, кагоцел, ацикловир"


In [36]:
df_diseases_final.to_csv('diseases_table.csv')